In [1]:
import numpy as np
import pickle
import scipy.linalg as linalg
import itertools
import math
from warnings import warn
from matplotlib import pyplot as plt

In [9]:
import numpy as np

# Example vector A
A = np.array([1, 2, 3, 4])

# Calculate the length of the resulting vector B
n = A.shape[0]
length_B = n * (n - 1) // 2
B = np.zeros(length_B, dtype=np.float64)

# Reshape A for broadcasting
A_ = A[:, np.newaxis]
B = (A_ @ A_.T)[np.triu_indices(A.shape[0], k = 1)]

print(B)

[ 2  3  4  6  8 12]


In [4]:
np.set_printoptions(suppress=True,precision=3)
A = np.load('../data/cheetah-arsk-min-5/best_koopman_policy_weights.npy', allow_pickle = True)
print(A.round(3))

[[  0.971   2.731   4.343   3.672   0.659   3.797   0.225  -0.375   3.926
   -2.816   0.973   4.743  -0.379   2.016   1.199   2.557   1.986   3.154
    0.124   4.076   0.059  -1.188  -0.727  -0.11 ]
 [ -0.54   -2.6    -6.247   0.898  -0.159  -7.882   4.54   -0.823   4.388
    0.562   4.191   2.141   2.82    3.343  -8.078  -0.728   2.78   -1.363
    1.652  -0.206   5.206 -10.682   2.561  -2.317]
 [  2.6     6.207   0.285   3.44    0.656  -2.598  -3.102  -3.137   0.657
   -5.651   0.987   1.191  -3.217  -2.489   1.162   2.643  -0.388   0.004
   -0.703  -3.407   2.464   1.288   3.093   4.809]
 [  3.104   0.302  -9.262  -5.08   -4.003 -11.155   4.374  -8.703   6.101
   -0.098   1.644   8.331   3.985   4.937   2.982   4.355   0.385  -3.242
   -4.369   2.073   0.999   0.888  -7.234   1.652]
 [  0.672  -5.442  -2.612  -1.815   0.021  -1.504   7.935   5.468  -6.24
   -5.321 -12.081  -2.253  -0.53    3.062  -8.455  -0.326  -5.895   1.175
    4.041   2.518   0.638   0.907  -3.449  -2.906]
 [  4.

In [2]:
#load trajectories - each of shape (#successful episodes, lifted Z space, time rollout)
cube_trajectories = np.load(open('./expert_trajectories/cube.npy', 'rb'))
ball_trajectories = np.load(open('./expert_trajectories/ball.npy', 'rb'))
cylinder_trajectories = np.load(open('./expert_trajectories/cylinder.npy', 'rb'))
foam_brick_trajectories = np.load(open('./expert_trajectories/foam_brick.npy', 'rb'))

print(cube_trajectories.shape)
print(ball_trajectories.shape)
print(cylinder_trajectories.shape)
print(foam_brick_trajectories.shape)

(187, 759, 100)
(191, 759, 100)
(172, 759, 100)
(190, 759, 100)


In [6]:
# https://humaticlabs.com/blog/mrdmd-python/
def dmd(X, Y, truncate=None):
    if truncate == 0:
        # return empty vectors
        mu = np.array([], dtype='complex')
        Phi = np.zeros([X.shape[0], 0], dtype='complex')
    else:
        U2,Sig2,Vh2 = linalg.svd(X) # SVD of input matrix
        r = len(Sig2) if truncate is None else truncate # rank truncation
        U = U2[:,:r]
        Sig = np.diag(Sig2)[:r,:r]
        V = Vh2.conj().T[:,:r]
        Atil = np.dot(np.dot(np.dot(U.conj().T, Y), V), linalg.inv(Sig)) # build A tilde
        mu,W = linalg.eig(Atil)
        Phi = np.dot(np.dot(np.dot(Y, V), linalg.inv(Sig)), W) # build DMD modes
    return mu, Phi

def check_dmd_result(X, Y, mu, Phi, show_warning=True):
    b = np.allclose(Y, np.dot(np.dot(np.dot(Phi, np.diag(mu)), linalg.pinv(Phi)), X))
    if not b and show_warning:
        warn('dmd result does not satisfy Y=AX')

In [4]:
# https://humaticlabs.com/blog/mrdmd-python/
def svht(X, sv=None):
    # svht for sigma unknown
    m,n = sorted(X.shape) # ensures m <= n
    beta = m / n # ratio between 0 and 1
    if sv is None:
        sv = linalg.svdvals(X)
    sv = np.squeeze(sv)
    omega_approx = 0.56 * beta**3 - 0.95 * beta**2 + 1.82 * beta + 1.43
    return np.median(sv) * omega_approx

In [5]:
def mrdmd(D, level=0, bin_num=0, offset=0, max_levels=7, max_cycles=2, do_svht=True):
    """Compute the multi-resolution DMD on the dataset `D`, returning a list of nodes
    in the hierarchy. Each node represents a particular "time bin" (window in time) at
    a particular "level" of the recursion (time scale). The node is an object consisting
    of the various data structures generated by the DMD at its corresponding level and
    time bin. The `level`, `bin_num`, and `offset` parameters are for record keeping 
    during the recursion and should not be modified unless you know what you are doing.
    The `max_levels` parameter controls the maximum number of levels. The `max_cycles`
    parameter controls the maximum number of mode oscillations in any given time scale 
    that qualify as "slow". The `do_svht` parameter indicates whether or not to perform
    optimal singular value hard thresholding."""

    # 4 times nyquist limit to capture cycles
    nyq = 8 * max_cycles

    # time bin size
    bin_size = D.shape[1]
    if bin_size < nyq:
        return []

    # extract subsamples 
    step = math.floor(bin_size / nyq) # max step size to capture cycles
    _D = D[:,::step]
    X = _D[:,:-1]
    Y = _D[:,1:]

    # determine rank-reduction
    if do_svht:
        _sv = linalg.svdvals(_D)
        tau = svht(_D, sv=_sv)
        r = sum(_sv > tau)
    else:
        r = min(X.shape)

    # compute dmd
    mu,Phi = dmd(X, Y, r)

    # frequency cutoff (oscillations per timestep)
    rho = max_cycles / bin_size

    # consolidate slow eigenvalues (as boolean mask)
    slow = (np.abs(np.log(mu) / (2 * np.pi * step))) <= rho
    n = sum(slow) # number of slow modes

    # extract slow modes (perhaps empty)
    mu = mu[slow]
    Phi = Phi[:,slow]

    if n > 0:

        # vars for the objective function for D (before subsampling)
        Vand = np.vander(np.power(mu, 1/step), bin_size, True)
        P = np.multiply(np.dot(Phi.conj().T, Phi), np.conj(np.dot(Vand, Vand.conj().T)))
        q = np.conj(np.diag(np.dot(np.dot(Vand, D.conj().T), Phi)))

        # find optimal b solution
        b_opt = linalg.solve(P, q).squeeze()

        # time evolution
        Psi = (Vand.T * b_opt).T

    else:

        # zero time evolution
        b_opt = np.array([], dtype='complex')
        Psi = np.zeros([0, bin_size], dtype='complex')

    # dmd reconstruction
    D_dmd = np.dot(Phi, Psi)   

    # remove influence of slow modes
    D = D - D_dmd

    # record keeping
    node = type('Node', (object,), {})()
    node.level = level            # level of recursion
    node.bin_num = bin_num        # time bin number
    node.bin_size = bin_size      # time bin size
    node.start = offset           # starting index
    node.stop = offset + bin_size # stopping index
    node.step = step              # step size
    node.rho = rho                # frequency cutoff
    node.r = r                    # rank-reduction
    node.n = n                    # number of extracted modes
    node.mu = mu                  # extracted eigenvalues
    node.Phi = Phi                # extracted DMD modes
    node.Psi = Psi                # extracted time evolution
    node.b_opt = b_opt            # extracted optimal b vector
    nodes = [node]

    # split data into two and do recursion
    if level < max_levels:
        split = math.ceil(bin_size / 2) # where to split
        nodes += mrdmd(
            D[:,:split],
            level=level+1,
            bin_num=2*bin_num,
            offset=offset,
            max_levels=max_levels,
            max_cycles=max_cycles,
            do_svht=do_svht
            )
        nodes += mrdmd(
            D[:,split:],
            level=level+1,
            bin_num=2*bin_num+1,
            offset=offset+split,
            max_levels=max_levels,
            max_cycles=max_cycles,
            do_svht=do_svht
            )

    return nodes

In [6]:
def stitch(nodes, level):
    
    # get length of time dimension
    start = min([nd.start for nd in nodes])
    stop = max([nd.stop for nd in nodes])
    t = stop - start

    # extract relevant nodes
    nodes = [n for n in nodes if n.level == level]
    nodes = sorted(nodes, key=lambda n: n.bin_num)
    
    # stack DMD modes
    Phi = np.hstack([n.Phi for n in nodes])
    
    # allocate zero matrix for time evolution
    nmodes = sum([n.n for n in nodes])
    Psi = np.zeros([nmodes, t], dtype='complex')
    
    # copy over time evolution for each time bin
    i = 0
    for n in nodes:
        _nmodes = n.Psi.shape[0]
        Psi[i:i+_nmodes,n.start:n.stop] = n.Psi
        i += _nmodes
    
    return Phi,Psi

In [7]:
K = np.load('../hand_dapg/dapg/controller_training/koopman_without_vel/relocate/koopmanMatrix.npy')
print(K.shape)

(759, 759)


In [7]:
# reconstructed_data = np.dot(*stitch(nodes, i))

'''Generate new koopman matrices using DMD for each episode. Also store individual eigenvectors (modes) and eigenvalues'''

A = np.zeros((ball_trajectories.shape[0], ball_trajectories.shape[1], ball_trajectories.shape[1]))
Lambdas = np.zeros((ball_trajectories.shape[0], ball_trajectories.shape[2] - 1))
Eigvecs = np.zeros((ball_trajectories.shape[0], ball_trajectories.shape[1], ball_trajectories.shape[2] - 1))

for i in range(A.shape[0]):
    D = ball_trajectories[i]
    X, Y = D[:, :-1], D[:, 1:]

    Lambda, W = dmd(X, Y)
    Lambdas[i] = Lambda
    Eigvecs[i] = W
    check_dmd_result(X, Y, Lambda, W)

    #A = W @ Lambda @ W^-1
    #based on qualitative, this method approximates our actual K matrix better than truncating to real at the end? somehow...
    A[i] = (W @ np.diag(Lambda) @ linalg.pinv(W)).real

print(A.shape, Lambdas.shape, Eigvecs.shape)

/home/ctang306/miniconda3/envs/mjrl-env/lib/python3.7/site-packages/ipykernel_launcher.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  
/home/ctang306/miniconda3/envs/mjrl-env/lib/python3.7/site-packages/ipykernel_launcher.py:15: ComplexWarning: Casting complex values to real discards the imaginary part
  from ipykernel import kernelapp as app
/home/ctang306/miniconda3/envs/mjrl-env/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: dmd result does not satisfy Y=AX


(191, 759, 759) (191, 99) (191, 759, 99)


In [5]:
np.save('./k_mat_relocate/ball_K.npy', A)
np.save('./k_mat_relocate/ball_eigvecs.npy', Eigvecs)
np.save('./k_mat_relocate/ball_eigvals.npy', Lambdas)

In [9]:
eigvecs = np.load('./k_mat_relocate/ball_eigvecs.npy')
# print(eigvecs)
np.save('./k_mat_relocate/ball_eigvecs_first_episode.npy', eigvecs[0])

eigvals = np.load('./k_mat_relocate/ball_eigvals.npy')
print(eigvals[0])
np.save('./k_mat_relocate/ball_eigvals_first_episode.npy', eigvals[0])

A = np.load('./k_mat_relocate/ball_K.npy')
# print(A)
np.save('./k_mat_relocate/ball_K_first_episode.npy', A[0])

[-0.90695664 -0.9061085  -0.9061085  -0.90831032 -0.90831032 -0.89960583
 -0.89960583 -0.87975936 -0.87975936 -0.68456738 -0.68456738 -0.85429498
 -0.85429498 -0.82783275 -0.82783275  1.00729653  0.98866253  0.98866253
  0.97146065  0.97146065  0.9633386   0.9633386  -0.80997479 -0.80997479
  0.95332996  0.95332996  0.94387492  0.94387492  0.90741701  0.90741701
  0.87630292  0.87630292 -0.79114127 -0.79114127  0.86671186  0.86671186
  0.81925671  0.81925671  0.79286089  0.79286089 -0.73385791 -0.73385791
  0.75790807  0.75790807 -0.6825464  -0.6825464   0.71031003  0.71031003
 -0.63151181 -0.63151181  0.67055647  0.67055647 -0.57767954 -0.57767954
  0.6350265   0.6350265  -0.52037978 -0.52037978 -0.03635633  0.57532405
  0.57532405 -0.46046767 -0.46046767  0.52320594  0.52320594 -0.39795938
 -0.39795938 -0.33416838 -0.33416838  0.46361871  0.46361871 -0.27172124
 -0.27172124 -0.20876279 -0.20876279  0.4039742   0.4039742  -0.14509848
 -0.14509848 -0.0825645  -0.0825645   0.33954574  0

In [14]:
A_0 = A[0]
A_0_tilde = (eigvecs[0] @ np.diag(eigvals[0]) @ linalg.pinv(eigvecs[0]))
print(np.sum(np.abs(A_0_tilde - K)))
print(np.sum(np.abs(A_0 - K)))

133347.42428924816
160508.45346124517


In [ ]:
eigvecs = np.load('./k_mat_relocate/ball_eigvecs.npy')
# print(eigvecs)
np.save('./k_mat_relocate/ball_eigvecs_first_episode.npy', eigvecs[0])

eigvals = np.load('./k_mat_relocate/ball_eigvals.npy')
print(eigvals[0])
np.save('./k_mat_relocate/ball_eigvals_first_episode.npy', eigvals[0])

A = np.load('./k_mat_relocate/ball_K.npy')
# print(A)
np.save('./k_mat_relocate/ball_K_first_episode.npy', A[0])

[-0.90695664 -0.9061085  -0.9061085  -0.90831032 -0.90831032 -0.89960583
 -0.89960583 -0.87975936 -0.87975936 -0.68456738 -0.68456738 -0.85429498
 -0.85429498 -0.82783275 -0.82783275  1.00729653  0.98866253  0.98866253
  0.97146065  0.97146065  0.9633386   0.9633386  -0.80997479 -0.80997479
  0.95332996  0.95332996  0.94387492  0.94387492  0.90741701  0.90741701
  0.87630292  0.87630292 -0.79114127 -0.79114127  0.86671186  0.86671186
  0.81925671  0.81925671  0.79286089  0.79286089 -0.73385791 -0.73385791
  0.75790807  0.75790807 -0.6825464  -0.6825464   0.71031003  0.71031003
 -0.63151181 -0.63151181  0.67055647  0.67055647 -0.57767954 -0.57767954
  0.6350265   0.6350265  -0.52037978 -0.52037978 -0.03635633  0.57532405
  0.57532405 -0.46046767 -0.46046767  0.52320594  0.52320594 -0.39795938
 -0.39795938 -0.33416838 -0.33416838  0.46361871  0.46361871 -0.27172124
 -0.27172124 -0.20876279 -0.20876279  0.4039742   0.4039742  -0.14509848
 -0.14509848 -0.0825645  -0.0825645   0.33954574  0

In [16]:
# reconstructed_data = np.dot(*stitch(nodes, i))

'''Generate new koopman matrices using DMD for each episode. Also store individual eigenvectors (modes) and eigenvalues'''

# B = np.zeros((ball_trajectories.shape[0], ball_trajectories.shape[1], ball_trajectories.shape[1]))
# mr_eigvals = np.zeros((ball_trajectories.shape[0], ball_trajectories.shape[2] - 1))
# mr_eigvecs = np.zeros((ball_trajectories.shape[0], ball_trajectories.shape[1], ball_trajectories.shape[2] - 1))
# mr_node_list = []

# for i in range(B.shape[0]):
#     D = ball_trajectories[i]

#     nodes = mrdmd(D)
#     mr_node_list.append(nodes)
#     # D_mrdmd = np.array([np.dot(*stitch(nodes, i)) for i in range(3)])
#     # B[i] = np.sum(D_mrdmd, axis = 0)

#     # eigvals, eigvecs = linalg.eig(B[i])
#     # print(eigvals, eigvecs)

# mr_node_list = np.array(mr_node_list)
# print(B.shape, mr_eigvals.shape, mr_eigvecs.shape)

(191, 759, 759) (191, 99) (191, 759, 99)


In [25]:
# first_ep_nodes = mr_node_list[0]
# sum_tmp_K = np.zeros(K.shape)
# sum_tmp_K_real = np.zeros(K.shape)

# print(first_ep_nodes.shape)
# for node in first_ep_nodes:
#     phi, mu = mr_node_list[0, 3].Phi, mr_node_list[0, 3].mu
#     sum_tmp_K += (phi @ np.diag(mu) @ linalg.pinv(phi)).real
#     sum_tmp_K_real += (phi.real @ np.diag(mu.real) @ linalg.pinv(phi).real)

# print(np.sum(np.abs(sum_tmp_K - K)))
# print(np.sum(np.abs(sum_tmp_K_real - K)))
# print(np.sum(np.abs(sum_tmp_K - sum_tmp_K_real)))

(7,)
128699.4496664777
128730.74651994009
597.5184205021764
